# [GD-08] Seq2seq
"Going Deeper Node 8. Making translator using Seq2seq" / 2022. 04. 07 (Thu) 이형주

## Contents
---
- **1. Environment Setup**
- **2. Modeling**
- **3. Project Retrospective**

## Rubric 평가기준
---

|  평가문항  |  상세기준  |
|:---------|:---------|
|1. 번역기 모델 학습에 필요한 텍스트 데이터 전처리가 한국어 포함하여 잘 이루어졌다.|구두점, 대소문자, 띄어쓰기, 한글 형태소분석 등 번역기 모델에 요구되는 전처리가 정상적으로 진행되었다.
|2. Attentional Seq2seq 모델이 정상적으로 구동된다.|seq2seq 모델 훈련 과정에서 training loss가 안정적으로 떨어지면서 학습이 진행됨이 확인되었다.
|3. 테스트 결과 의미가 통하는 수준의 번역문이 생성되었다.|테스트용 디코더 모델이 정상적으로 만들어져서, 정답과 어느 정도 유사한 영어 번역이 진행됨을 확인하였다.

In [1]:
# Colab Pro GPU 정보
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Apr  7 14:31:47 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# Colab Pro Ram 정보
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


## 1. Environment Setup

In [3]:
import tensorflow as tf
import numpy as np
import time
import re
import os
import io
import random
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split

In [7]:
import tarfile
import urllib

# 데이터가 존재하는 웹 주소
DOWNLOAD_ROOT = "https://github.com/jungyeul/korean-parallel-corpora/blob/master/korean-english-news-v1/"
# 다운받을 정상/스팸 파일 명
DEV_URL = DOWNLOAD_ROOT + "korean-english-park.dev.tar.gz"
TEST_URL = DOWNLOAD_ROOT + "korean-english-park.test.tar.gz"
TRAIN_URL = DOWNLOAD_ROOT + "korean-english-park.train.tar.gz"
# 파일을 저장할 로컬 디렉토리 경로
DATA_PATH = os.path.join("datasets")

In [15]:
def fetch_data_path(dev_path=DEV_URL, test_path=TEST_URL, train_path=TRAIN_URL):
    if not os.path.isdir(DATA_PATH): # 디렉토리 존재 유무 확인
        os.makedirs(DATA_PATH)
    for filename, url in (("korean-english-park.dev.tar.gz", DEV_URL), 
                          ("korean-english-park.test.tar.gz", TEST_URL), 
                          "korean-english-park.train.tar.gz", TRAIN_URL):
        path = os.path.join(DATA_PATH, filename)
        if not os.path.isfile(path): # 파일 존재 유무 확인
            # 파일이 없으면 path로 전달한 경로로 url에 해당하는 파일을 저장
            urllib.request.urlretrieve(url, path)
            
        # 압축해제
        tar = tarfile.open(path)
        tar.extractall(path=DATA_PATH)
        tar.close()

## 2. Modeling

## 3. Project Retrospective